In [1]:
import cv2
import numpy as np
import pandas as pd
import statistics
import torch
import os
import time
import matplotlib.pyplot as plt
import math
import torch.nn.functional as F
import torchvision.transforms as tf
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import make_grid
from PIL import Image
from collections import OrderedDict
import seaborn as sns

torch.cuda.empty_cache()


In [2]:
modelA = torch.load('MODELS/modelA_bw_resnet18_withTL.pth',map_location=torch.device('cuda:0'))
modelB = torch.load('MODELS/modelB_bw_resnet18_withTL.pth',map_location=torch.device('cuda:0'))

In [3]:
print(modelA)
print(modelB)

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [4]:
#full_core = cv2.imread('../Machine Learning_GARTH/Garth_base_oriented.jpg')
#full_core = cv2.cvtColor(full_core,cv2.COLOR_BGR2RGB)

Image.MAX_IMAGE_PIXELS = None

samples = ['Gulda_top_of_mid_orientedv2']
zooms=[0.25,0.5,1,2,4,8]

trans = tf.Compose([tf.Resize((256,256)),
                  tf.ToTensor(),
                   ])

In [5]:
#Slice the Image


In [7]:
#Feed in the image and collect results

In [ ]:
for sample in samples:
   
    for zoom in zooms:   
   
        full_core = Image.open(f'../Machine Learning_GARTH/201012_jpgs/{sample}.jpg')
        if len(full_core.getbands())>1:
            full_core = np.array(Image.open(f'../Machine Learning_GARTH/201012_jpgs/{sample}.jpg').getchannel('G'))
        else:
            full_core = np.array(Image.open(f'../Machine Learning_GARTH/201012_jpgs/{sample}.jpg'))


        h_in = int(256/zoom)
        w_in = h_in

        h = full_core.shape[0]
        w = full_core.shape[1]

        n_h = h//h_in
        n_w = w//w_in

        r_h = h%h_in
        r_w = w%w_in

        print(f'height: {h} pixels,width: {w} pixels')
        print(f'images in height: {n_h},images in width: {n_w}')
        print(f'lost pixels in height: {r_h}, lost pixels in width: {r_w}')



        full_core = full_core[int(round(r_h/2)):(h-int(round(r_h/2))),int(round(r_w/2)):(w-int(round(r_w/2)))]

        quality_matrix = np.zeros((n_h,n_w), dtype = int) 
        layer_matrix = np.zeros((n_h,n_w), dtype = float)

        w_point = 0
        start_time = time.time()

        for j in range(n_w):
            try:
                if j%int(n_w/10) ==0:
                    print(f'{j//int(n_w/10)*10:.0F}%')
            except:
                pass
            h_point = 0
            for i in range(n_h):        
                active_image = full_core[h_point:h_point+h_in,w_point:w_point+w_in]
                active_image = Image.fromarray(active_image)

                h_point+=h_in

                quality_matrix[i,j] =  modelA(trans(active_image).view(1,1,256,256).cuda()).argmax(dim=1).item()

                if quality_matrix[i,j]!=0:
                    layer_matrix[i,j] =  modelB(trans(active_image).view(1,1,256,256).cuda()).item()

            w_point+=w_in

        print(time.time()-start_time)

        #save model

        tosave = pd.DataFrame(layer_matrix)
        tosave.head()
        tosave.to_csv(f'{sample}_{zoom}.csv',index=False, header=True)


        #load model
        toload=pd.read_csv(f'{sample}_{zoom}.csv')
        layer_matrix1 = toload.to_numpy()
        layer_matrix= layer_matrix1

        full_core_density_bin = np.zeros((full_core.shape[0],full_core.shape[1]), dtype = float)

        for j in range(full_core_density_bin.shape[1]):
            for i in range(full_core_density_bin.shape[0]):
                full_core_density_bin[i,j]=layer_matrix[(i-1)//h_in,(j-1)//w_in]

            if j%int(full_core_density_bin.shape[1]/10)==0:
                print(f'{j/int(full_core_density_bin.shape[1])*100:.0F}%')

        fig,(ax1,ax2,ax3) = plt.subplots(nrows=3,ncols=1, sharex=False)

        im = ax2.imshow(full_core_density_bin,cmap='hot')
        fig.colorbar(im, ax = ax3, orientation='horizontal')

        #ax1.set_xticklabels()
        fig.set_figheight(8)
        fig.set_figwidth(15)
        ax1.imshow(full_core, cmap='gray')
        ax1.set_yticks([])
        ax2.set_yticks([])
        ax1.set_xticks(np.round(np.linspace(0,full_core.shape[1],10)))
        #ax1.set_xticklabels(np.linspace(0,24251,10,dtype=int))

        ax2.set_xticks(np.round(np.linspace(0,full_core.shape[1],10)))
        #ax2.set_xticklabels(np.linspace(0,24251,10,dtype=int))
        ax2.set_xlabel('position in sample (in pixels)')
        ax3.imshow(np.zeros([1,full_core.shape[1]]))
        ax3.set_yticks([])
        ax3.set_xticks([])
        ax3.set_xticklabels([])
        ax3.set_axis_off()

        stamp = time.strftime("%Y%m%d%H%M%S", time.gmtime())

        plt.savefig('core_layerDensity_{}_{}_{}.png'.format(sample,zoom, str(stamp)))


        carlo = []
        layer_reduced=[]
        sd=[]
        sem=[]
        avg=[]
        n=[]
        missing_len=0
        missing_tot=0

        for j in range(layer_matrix.shape[1]-1):


            popul = layer_matrix[:,j][layer_matrix[:,j]!=0]
           # try: 
            layer_reduced.append(popul)
            if popul.size == 0:
                missing_len+=1
            else:
                missing_tot+=popul.size

           # except:
           #     layer_reduced.append(100000)
           #     print(10000000000000000)

            try:
                sd.append(np.std(popul))       
            except:
                sd.append(0)
            try:
                avg.append(np.mean(popul))
            except:
                avg.append(0)


            try:    
                n.append(popul.size)
            except:
                n.append(0)

            try:
                sem.append(np.std(popul)/np.sqrt(popul.size))
            except:
                sem.append(0)





        sd.append(0)
        sem.append(0)
        avg.append(0)
        n.append(0)

        missing_len = missing_len/n_w*100
        missing_tot = 1-(missing_tot/(n_w*n_h))


        for i in range(10000):

            if i%1000 ==0:
                print(f'{i//1000*10:.0F}%')

            carlo_count = 0
            for popul in layer_reduced:
    #            popul = layer_matrix[:,j][layer_matrix[:,j]!=0]
    #            popul = layer_matrix[:,j]

                try:
                    carlo_count += np.random.choice(popul,1,replace=False).item()
                except:
                    pass
            carlo.append(carlo_count)

        minimum_carlo = round(min(carlo))
        maximum_carlo = round(max(carlo))
        avg_carlo = round(statistics.mean(carlo))
        med_carlo = round(statistics.median(carlo))
        sd_carlo=round(statistics.stdev(carlo))
        stepsize_carlo = int((maximum_carlo-minimum_carlo)/20) if int((maximum_carlo-minimum_carlo)/20)>0 else 1


        fig,(ax1) = plt.subplots(nrows=1,ncols=1, sharex=False)
        fig.set_figheight(8)
        fig.set_figwidth(8)

        ax1.hist(carlo,bins=(range(minimum_carlo,maximum_carlo+stepsize_carlo,stepsize_carlo)))
        ax1.axvline(avg_carlo, color='k', linestyle='dashed', linewidth=1)
        ax1.text(avg_carlo+stepsize_carlo, stepsize_carlo, 'Mean: {:.2f}'.format(avg_carlo))

        ax1.set_xlabel('n total layers in sample')
        ax1.set_ylabel('count of Monte Carlo (10000)')
        ylim=ax1.get_ylim()[1]*1.5
        ax1.set_ylim(0,ylim)

        ax1.text(minimum_carlo-stepsize_carlo/4,ylim*0.70,f'''Number of recombinations: 10,000
        Sample Name: {sample}
        Zoom: {zoom}x
        unconstrained sample:{missing_tot:.1F}%
        unconstrained length of sample:{missing_len:.1F}%
        Minimmun number of layers: {minimum_carlo}
        Maximum number of layers: {maximum_carlo}
        Average number of layers:{avg_carlo}
        Median number of layers: {med_carlo}
        SD of the simulation: {sd_carlo}''')




        stamp = time.strftime("%Y%m%d%H%M%S", time.gmtime())

        plt.savefig('monte_carlo_{}_{}_{}.png'.format(sample, zoom, str(stamp)))

        plt.show()



height: 4408 pixels,width: 24862 pixels
images in height: 4,images in width: 24
lost pixels in height: 312, lost pixels in width: 286
0%
